# **Install Libraries**

In [2]:
!pip install "ray[tune]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **Import Libraries**

In [3]:
from filelock import FileLock
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from torch.utils.data import random_split
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from collections import OrderedDict

# **Data Loaders**

In [4]:
def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    with FileLock(os.path.expanduser("~/.data.lock")):
        trainset = torchvision.datasets.CIFAR10(
            root=data_dir, train=True, download=True, transform=transform
        )

        testset = torchvision.datasets.CIFAR10(
            root=data_dir, train=False, download=True, transform=transform
        )

    return trainset, testset

# **Configurable Neural Network**

In [5]:
def flatten(x, start_dim=1, end_dim=-1):
    return x.flatten(start_dim=start_dim, end_dim=end_dim)

In [6]:
# Will need to modify the parameters and the network depending on what you are experimenting with

C, H, W = 3, 32, 32
num_classes = 10

channel_1 = 16
channel_2 = 32
channel_3 = 64

kernel_size_1 = 3
kernel_size_2 = 3
kernel_size_3 = 6

pad_size_1 = 2
pad_size_2 = 1
pad_size_3 = 1

pool_kernel_size = 2

fc_count_1 = 256

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, channel_1, kernel_size_1, padding=pad_size_1)
        self.norm1 = nn.BatchNorm2d(channel_1)
        self.pool1 = nn.MaxPool2d(pool_kernel_size)
        self.conv2 = nn.Conv2d(channel_1, channel_2, kernel_size_2, padding=pad_size_2)
        self.norm2 = nn.BatchNorm2d(channel_2)
        self.pool2 = nn.MaxPool2d(pool_kernel_size)
        self.conv3 = nn.Conv2d(channel_2, channel_3, kernel_size_3, padding=pad_size_3)
        self.norm3 = nn.BatchNorm2d(channel_3)
        self.pool3 = nn.MaxPool2d(pool_kernel_size)        

        H_after_conv1 = int(H + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        W_after_conv1 = int(W + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        H_after_pool1 = int((H_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool1 = int((W_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        H_after_conv2 = int(H_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        W_after_conv2 = int(W_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        H_after_pool2 = int((H_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool2 = int((W_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)  
        H_after_conv3 = int(H_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        W_after_conv3 = int(W_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        H_after_pool3 = int((H_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool3 = int((W_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)

        self.fc1 = nn.Linear(H_after_pool3*W_after_pool3*channel_3, fc_count_1)
        self.fc2 = nn.Linear(fc_count_1, fc_count_1)
        self.fc3 = nn.Linear(fc_count_1, 10)


    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = F.relu(x)
        x = self.pool3(x)
        x = flatten(x)
        x = self.fc1(x)
        #x = F.relu(x)
        x = self.fc2(x)
        #x = F.relu(x)
        x = self.fc3(x)

        return x

Train using SGD optimizer

In [7]:
def train_cifar(config):
    net = Net()  # Will need to modify the parameters depending on what you are experimenting with

    learning_rate = config['learning_rate']
    momentum = config['momentum']
    weight_decay = config['weight_decay']

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    optimizer = None

    criterion = nn.CrossEntropyLoss()

    if config['opt'] == 'SGD':
      if momentum == 0: 
        optimizer = optim.SGD(net.parameters(), lr=learning_rate, 
                      weight_decay=weight_decay,
                      momentum=0) 
      else: 
        optimizer = optim.SGD(net.parameters(), lr=learning_rate, 
                      weight_decay=weight_decay, 
                      momentum=momentum, nesterov=True)  
    elif config['opt'] == 'Adagrad':
      optimizer = optim.Adagrad(net.parameters(), lr=learning_rate, 
                    weight_decay=weight_decay)  
    elif config['opt'] == 'RMSprop':
      optimizer = optim.RMSprop(net.parameters(), lr=learning_rate, 
                    weight_decay=weight_decay, momentum=momentum)
    elif config['opt'] == 'Adam':
      optimizer = optim.Adam(net.parameters(), lr=learning_rate, 
                    weight_decay=weight_decay)

    # Do not modify any code below!
    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state = torch.load(
                os.path.join(loaded_checkpoint_dir, "checkpoint.pt")
            )
            net.load_state_dict(model_state)
            optimizer.load_state_dict(optimizer_state)

    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=64, shuffle=True, num_workers=8
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=64, shuffle=True, num_workers=8
    )

    for epoch in range(10):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:
                print(
                    f"[{epoch + 1}, {i + 1:>5} loss: {running_loss / epoch_steps:.3f}]"
                )
                running_loss = 0.0

        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        os.makedirs("my_model", exist_ok=True)
        torch.save((net.state_dict(), optimizer.state_dict()), "my_model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("my_model")
        session.report(
            {"loss": (val_loss / val_steps), "accuracy": correct / total},
            checkpoint=checkpoint,
        )
    print("Finished Training")

# **Test Set Accuracy**

In [8]:
def test_best_model(best_result):
    best_trained_model = Net(
        best_result.config["learning_rate"], 
        best_result.config["weight_decay"]
     )  # Will need to modify the parameters depending on what you are experimenting with

    # Do not modify any code below!
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(
        best_result.checkpoint.to_directory(), "checkpoint.pt"
    )

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Best trial test set accuracy: {correct / total}")

# **Configuring the Search Space**

In [9]:
# You will be experimenting with the hyperparameters here
# Use tune.grid_search to specify what values you want to experiment with a hyperparameter

#with momentum
config = {"opt": tune.grid_search(['SGD', 'Adagrad', 'RMSprop', 'Adam']),
          "learning_rate": tune.grid_search([1e-3, 1e-2, 1e-1, 1]), 
          "momentum": tune.grid_search([0, 0.5, 0.9]), 
          "weight_decay": tune.grid_search([1e-4, 1e-3, 1e-2, 1e-1])}

In [10]:
# Do not modify any code below!
def main(config):
    #SGD
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_cifar),
            resources={"gpu": 1},
        ),
        param_space=config,
    )
    results = tuner.fit()
    best_result = results.get_best_result("accuracy", "max")

    print(f"Best trial config: {best_result.config}")
    print(f"Best trial final validation loss: {best_result.metrics['loss']}")
    print(f"Best trial final validation accuracy: {best_result.metrics['accuracy']}")

    test_best_model(best_result)

    return results.get_dataframe("accuracy", "max")


results_df = main(config)

2023-04-13 05:37:21,786	INFO worker.py:1553 -- Started a local Ray instance.


(train_cifar pid=432366) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-13_05-37-19/train_cifar_43ecb_00000_0_learning_rate=0.0010,momentum=0,opt=SGD,weight_decay=0.0001_2023-04-13_05-37-23/data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:03<00:00, 49469591.59it/s]


(train_cifar pid=432366) Extracting /root/ray_results/train_cifar_2023-04-13_05-37-19/train_cifar_43ecb_00000_0_learning_rate=0.0010,momentum=0,opt=SGD,weight_decay=0.0001_2023-04-13_05-37-23/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-13_05-37-19/train_cifar_43ecb_00000_0_learning_rate=0.0010,momentum=0,opt=SGD,weight_decay=0.0001_2023-04-13_05-37-23/data
(train_cifar pid=432366) Files already downloaded and verified


Trial name,accuracy,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_cifar_43ecb_00000,0.5574,2023-04-13_05-38-22,True,,71e4524af6144c99acaef042b08b0996,"0_learning_rate=0.0010,momentum=0,opt=SGD,weight_decay=0.0001",d72b2a1ba8c6,10,1.23951,172.28.0.12,432366,True,56.0189,4.53156,56.0189,1681364302,0,,10,43ecb_00000,0.00344133
train_cifar_43ecb_00001,0.694,2023-04-13_05-39-14,True,,71e4524af6144c99acaef042b08b0996,"1_learning_rate=0.0100,momentum=0,opt=SGD,weight_decay=0.0001",d72b2a1ba8c6,10,0.929891,172.28.0.12,432366,True,52.3032,4.46493,52.3032,1681364354,0,,10,43ecb_00001,0.00344133
train_cifar_43ecb_00002,0.7129,2023-04-13_05-40-05,True,,71e4524af6144c99acaef042b08b0996,"2_learning_rate=0.1000,momentum=0,opt=SGD,weight_decay=0.0001",d72b2a1ba8c6,10,0.972429,172.28.0.12,432366,True,50.2261,4.5072,50.2261,1681364405,0,,10,43ecb_00002,0.00344133
train_cifar_43ecb_00003,0.1032,2023-04-13_05-40-55,True,,71e4524af6144c99acaef042b08b0996,"3_learning_rate=1,momentum=0,opt=SGD,weight_decay=0.0001",d72b2a1ba8c6,10,nan,172.28.0.12,432366,True,50.4809,4.44844,50.4809,1681364455,0,,10,43ecb_00003,0.00344133
train_cifar_43ecb_00004,0.6372,2023-04-13_05-41-48,True,,71e4524af6144c99acaef042b08b0996,"4_learning_rate=0.0010,momentum=0.5000,opt=SGD,weight_decay=0.0001",d72b2a1ba8c6,10,1.0282,172.28.0.12,432366,True,52.5742,4.59996,52.5742,1681364508,0,,10,43ecb_00004,0.00344133
train_cifar_43ecb_00005,0.711,2023-04-13_05-42-39,True,,71e4524af6144c99acaef042b08b0996,"5_learning_rate=0.0100,momentum=0.5000,opt=SGD,weight_decay=0.0001",d72b2a1ba8c6,10,0.924305,172.28.0.12,432366,True,51.4171,4.51451,51.4171,1681364559,0,,10,43ecb_00005,0.00344133
train_cifar_43ecb_00006,0.7131,2023-04-13_05-43-32,True,,71e4524af6144c99acaef042b08b0996,"6_learning_rate=0.1000,momentum=0.5000,opt=SGD,weight_decay=0.0001",d72b2a1ba8c6,10,0.962541,172.28.0.12,432366,True,52.2941,4.70528,52.2941,1681364612,0,,10,43ecb_00006,0.00344133
train_cifar_43ecb_00007,0.1042,2023-04-13_05-44-22,True,,71e4524af6144c99acaef042b08b0996,"7_learning_rate=1,momentum=0.5000,opt=SGD,weight_decay=0.0001",d72b2a1ba8c6,10,nan,172.28.0.12,432366,True,50.5902,4.46949,50.5902,1681364662,0,,10,43ecb_00007,0.00344133
train_cifar_43ecb_00008,0.702,2023-04-13_05-45-13,True,,71e4524af6144c99acaef042b08b0996,"8_learning_rate=0.0010,momentum=0.9000,opt=SGD,weight_decay=0.0001",d72b2a1ba8c6,10,0.914767,172.28.0.12,432366,True,50.551,4.65169,50.551,1681364713,0,,10,43ecb_00008,0.00344133
train_cifar_43ecb_00009,0.7129,2023-04-13_05-46-05,True,,71e4524af6144c99acaef042b08b0996,"9_learning_rate=0.0100,momentum=0.9000,opt=SGD,weight_decay=0.0001",d72b2a1ba8c6,10,1.01093,172.28.0.12,432366,True,52.1122,4.48838,52.1122,1681364765,0,,10,43ecb_00009,0.00344133


...

100%|██████████| 170498071/170498071 [00:08<00:00, 19138691.30it/s]


(train_cifar pid=432366) Extracting /root/ray_results/train_cifar_2023-04-13_05-37-19/train_cifar_43ecb_00191_191_learning_rate=1,momentum=0.9000,opt=Adam,weight_decay=0.1000_2023-04-13_08-36-44/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-13_05-37-19/train_cifar_43ecb_00191_191_learning_rate=1,momentum=0.9000,opt=Adam,weight_decay=0.1000_2023-04-13_08-36-44/data
(train_cifar pid=432366) Files already downloaded and verified
(train_cifar pid=432366) Finished Training


2023-04-13 08:37:49,286	INFO tune.py:798 -- Total run time: 10826.30 seconds (10826.12 seconds for the tuning loop).


Best trial config: {'opt': 'Adagrad', 'learning_rate': 0.01, 'momentum': 0.5, 'weight_decay': 0.01}
Best trial final validation loss: 0.7653940382656778
Best trial final validation accuracy: 0.7282


TypeError: ignored